# Figure 2 (top)

This notebook generates Figure 2 (top) from simulation results stored in 'data/figure_2_top_source.mat'.

## Imports

In [ ]:
import os
os.chdir('..')

import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import scipy.io as scio

## Data

The figure is generated from files generated by `SIDARTHE/SIDARTHE_policy_distribution/NotebookData.m`:

 - `data/figure_2_top_peaks.mat` contains, for each simulated combination of working days and quarantine days, the number of infected individuals at the peak of the epidemic post-lockdown (`peak`) and the time at which this peak occured (`peakTime`).
 - `data/figure_2_top_timeseries.mat` contains, for a selected subset of FPSP policies, timeseries of infected individuals.

In [ ]:
peak_file = 'data/figure_2_top_peaks.mat'
sim_file = 'data/figure_2_top_timeseries.mat'
mat = scio.loadmat(peak_file)
sims = scio.loadmat(sim_file)

def peak(x, y):
    return mat['peak'][x,y]

def peakTime(x, y):
    return mat['peakTime'][x, y]

def get_sim(idx):
    x = sims['TimeSeries'][idx][0].reshape(-1)[0]
    y = sims['TimeSeries'][idx][1].reshape(-1)[0]
    ts = sims['TimeSeries'][idx][2].reshape(-1)
    ys = sims['TimeSeries'][idx][3]
    infecteds = np.sum(ys[:,2:7], axis=1)
    return x, y, ts, infecteds

## Figure

In [ ]:
# setup
cmap = plt.get_cmap('inferno') # perceptually uniform colormap
plt.rcParams.update({'font.size': 16})

# increases the dynamic range of colors by mapping duty cycles to interval [0, 1]
def rescale(x):
    min_x = 0.07
    max_x = 0.5
    return (x-min_x)/(max_x-min_x)

In [ ]:
""" 
layout
"""
fig = plt.figure(figsize=(30,10))
spec = gridspec.GridSpec(ncols=3, nrows=4, figure=fig)
tl = fig.add_subplot(spec[:4,0]) # top left figure spans across all rows
tr = fig.add_subplot(spec[:4,2]) # top right figure spans across all rows
tm = [fig.add_subplot(spec[i, 1]) for i in range(4)] # top middle contains one figure per row

"""
top left and top right subplots
"""
# plot period lengths up to 112 days that are multiples of 7 days and 14 days, respectively.
ax = tl
for base_period in [7, 14]:
    multipliers = [(i+1) for i in range(int(112/base_period))]
    periods = [base_period*i for i in multipliers]
    
    # plot simulation results only for a subset of duty cycles
    for duty_int in range(1,int(base_period/2)+1):
        duty_cycle = duty_int/base_period
        
        # skip repetitions
        if base_period > 7 and (duty_int % (base_period/7) == 0):
            continue
        if base_period >= 14 and (duty_int >= 7):
            continue
            
        xs = [duty_int * i for i in multipliers]
        ys = [(base_period-duty_int)*i for i in multipliers]
        peak_values = [peak(x, y) for x, y in zip(xs, ys)]
        peak_times = [peakTime(x, y) for x, y in zip(xs, ys)]
        color = cmap(rescale(duty_cycle))
        tl.semilogy(np.array(periods)/7, np.array(peak_values)/1e5, 'o-', c=color)
        tr.plot(np.array(periods)/7, peak_times, 'o-', c=color, label=': {:.0f}% duty-cycle, e.g. (X={},Y={})'.format(duty_cycle*100, duty_int, (base_period-duty_int)))

# design settings: top left
ax = tl
ax.grid('on', axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
y_ticks = [1, 2, 5, 10, 20, 50]
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks)
ax.set_ylabel('Peak Infected (% of 10 million)')
ax.set_xlabel('Period Length (weeks)')

# design settings: top right
ax = tr
ax.grid('on', axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
#ax.spines['left'].set_visible(False)
ax.legend()
#ax.set_ylabel('Peak Value (percentage of 10 million)')
ax.set_ylabel('Peak Time (days)')
ax.set_xlabel('Period Length (weeks)')

plt.tight_layout()

"""
top center subplot
"""
# layout
ax = tm
ylim = [0, 60]
xlim = [0, 400]
xmarkers = [0, 100, 200, 300, 400]
# inset layout
zoom = (2.75,1.25)
axins = [inset_axes(ax[i], zoom[0], zoom[1] , loc='upper right')  for i in range(len(ax))]
inxlim = (0, 155)
inylim = (0.1, 2.2)

xs = [i+1 for i in range(6)]
ys = [14-x for x in xs]
multiples = [1, 2, 4, 6, 8]
t_max=1600
for i in reversed(range(4)):
    multiple = multiples[i]
    base_idx = i*len(xs)
    for j in xs[:6]:
        x, y, t, infecteds = get_sim(base_idx+j-1)
        duty_cycle = x/(x+y)
        ax[i].plot(t[:t_max], infecteds[:t_max]/1e5, c=cmap(rescale(duty_cycle)))
        axins[i].plot(t[:t_max], infecteds[:t_max]/1e5, c=cmap(rescale(duty_cycle)))
    
    t_start = np.argmax(t>=50)
    ax[i].plot([50, 50], ylim,'--', c='black', label='FPSP start', alpha=0.25)
    ax[i].set_ylim(ylim)
    ax[i].set_xlim(xlim)
    axins[i].plot([50, 50], ylim,'--', c='black', label='FPSP start', alpha=0.25)
    axins[i].set_xlim(inxlim[0], inxlim[1])
    axins[i].set_ylim(inylim[0], inylim[1])
    axins[i].grid(axis='y')
    mark_inset(ax[i], axins[i], loc1=2, loc2=4, fc="none", ec="0.75")
    
    ax[i].set_ylabel('Infected (%)')
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    ax[i].grid(axis='y')
    if i < 3:
        ax[i].set_xticklabels([])
        axins[i].set_xticklabels([])
    #ax[i].spines['top'].set_visible(False)
    ax[i].set_title('Period Length: {} weeks'.format(int((x+y)/7)))
    
ax[-1].axis('on')
ax[-1].set_xticks(xmarkers)
ax[-1].set_xticklabels(xmarkers)
ax[-1].set_xlabel('Time (days)')
ax[0].legend(loc=2)

#plt.savefig('f2_3_column_final.eps', dpi=1200)
#plt.savefig('f2_3_column_final.png', dpi=300)